<a href="https://colab.research.google.com/github/kridtapon/Gravity-Reversion-Oscillator-GRO-/blob/main/Gravity_Reversion_Oscillator_(GRO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.5 MB/s eta 0:00:00


In [ ]:
pip install --upgrade yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# === Relative Volatility Index (RVI) ===
def calculate_rvi(df, period=14):
    close = df['Close']
    std = close.rolling(window=period).std()
    delta = std.diff()

    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)

    rvi = 100 * up.rolling(window=period).mean() / (up.rolling(window=period).mean() + down.rolling(window=period).mean())
    return rvi

# === Center of Gravity (COG) ===
def calculate_cog(df, period=10):
    prices = df['Close']
    weights = np.arange(1, period + 1)
    def cog_calc(window):
        num = np.sum(weights * window[::-1])
        denom = np.sum(window)
        return -num / denom if denom != 0 else 0
    return prices.rolling(window=period).apply(cog_calc, raw=True)

# === Download data ===
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'
df = yf.download(symbol, start=start_date, end=end_date, multi_level_index=False)

# === Indicator Calculations ===
df['RVI'] = calculate_rvi(df, period=14)
df['COG'] = calculate_cog(df, period=10)

# === Strategy Conditions ===
df['COG_Slope'] = df['COG'] - df['COG'].shift(10)

df['Entry'] = (df['RVI'] < 30) & (df['COG_Slope'] > 0)
df['Exit'] = (df['RVI'] > 70) & (df['COG_Slope'] < 0)

# === Filter date range ===
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# === Shift entries/exits to next bar ===
shifted_entries = df['Entry'].shift(1).fillna(False).astype(bool).to_numpy()
shifted_exits = df['Exit'].shift(1).fillna(False).astype(bool).to_numpy()

# === Run Backtest ===
portfolio = vbt.Portfolio.from_signals(
    close=df['Open'],
    entries=shifted_entries,
    exits=shifted_exits,
    init_cash=100_000,
    fees=0.001,
    slippage=0.002,
    freq='D'
)

# === Output ===
print(portfolio.stats())
portfolio.plot().show()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
<ipython-input-3-c7aa0244dfa3>:48: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  shifted_entries = df['Entry'].shift(1).fillna(False).astype(bool).to_numpy()
<ipython-input-3-c7aa0244dfa3>:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  shifted_exits = df['Exit'].shift(1).fillna(False).astype(bool).to_numpy()


Start                                2020-01-02 00:00:00
End                                  2024-12-31 00:00:00
Period                                1258 days 00:00:00
Start Value                                     100000.0
End Value                                  137162.509612
Total Return [%]                                37.16251
Benchmark Return [%]                           187.56104
Max Gross Exposure [%]                             100.0
Total Fees Paid                              5831.344891
Max Drawdown [%]                               61.646081
Max Drawdown Duration                  830 days 00:00:00
Total Trades                                          24
Total Closed Trades                                   24
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   58.333333
Best Trade [%]                                 36.952567
Worst Trade [%]                